### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Fall 2018 ###

# Financial Factor Analysis #

### Tyler Behle

   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Email: tbehle@wisc.edu  
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Student ID Number: 9072599682  
### Mitchel Berg
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Email: mberg9@wisc.edu  
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Student ID Number: 9072092852  
### Christian Colomb
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Email: ccolumb@wisc.edu  
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Student ID Number: 9074912750  
### Mike Osmian
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Email: osmian@wisc.edu  
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Student ID Number: 9073922677  

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##


The goal of our project is to use various regression techinques to determine the most indicative subset of financial factors that influence returns. In fields such as quantitative finance, this is a very important part in creating automated trading algorithms because the most predictive factors in an asset price allow opportunities for profit generation. To accomplish the goal of finding the most predictive subset of factors in an asset's returns we first generated synthetic data that simulates the performance of an asset over time. Then, we used this data in multiple different regression models to find the optimal subset that influences returns. Next, we will outline the process for synthetically generating data in which our models use. 

### 1.1. Data Creation

The first step in this project was to create synthetically generated time series data for factor values and their corresponding returns for our model to use. The factor and return values are represented as a percentage change per time step. This time step can be pictured as any increment in a series such as minute, hour or day. We decided to generate one hundred different factor values and two thousand time steps to base our models upon. Below are the decisions and assumptions made when creating this data:

* **Factors**: There are one hundred different factors and each factor's percentage change over a time step follows a normal distribution where the  standard deviation is sampled from the sum of both normal and uniform distributions because it allows for greater variability and potential for extreme values. 

<br>

* **Percent Changes:** Every factor has a corresponding list of percentage changes at each time step. In order to make our data more representative, we built covariance between factors by randomly choosing subsets of factors and weighting them together. We also decided to add random periodic jumps in our percentage change values to represent the idiosyncratic jumps that a market creates. We did this by iterating through all of the factors and choosing random time ticks via a poisson sample and adding a value sampled from the normal distribution.

<br>

* **Returns:** The percentage change in the return column is calculated as a linear combination of the percentage changes for each factor. We sampled a value from a normal distribution for each weight assocuiated to each factor component to calculate the return. The standard deviation used to sample from the normal distribution where we get the weight increases with the factors. Here heatmap of the correlation matrix for the returns and the factors.


INSERT IMAGE


We created a python script that generates the synthetic data and outputs a csv file that can be read into our Julia code. Below is the detailed and commented python script, note that it will not run in this notebook due to it being python code. 





## 2. Mathematical model ##




### Steve's Notes

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$\begin{bmatrix}
      1 & 2 \\
       3 & 4
    \end{bmatrix}
    \begin{bmatrix} x \\ y \end{bmatrix} =
    \begin{bmatrix} 5 \\ 6 \end{bmatrix}$$

And here is an example of an optimization problem in standard form:
$$\begin{aligned}
  \underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
    \text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
    & h_j(x) = 0 && j=1,\dots,r
    \end{aligned}$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

### 2.1. Model Overview

Our problem's model is linear.  It analyzes 100 financial factors to establish the best combination of factors to observe when investing in a company.  In order to most effectively track our model's results we generated and minipulated parts of our data set to see how the model reacts to changes and relationships between factor data.

### 2.2. Parameters

$$\begin{aligned}
    \text{Parameters:}\qquad& \lambda = 0.01 && \text{Regularization Tradeoff Parameter}\\
    & \mu = 0.00001 && \text{Regularization Tradeoff Parameter}\\
    & n = 100 && \text{Number of Factors}\\
    & d = 10000 && \text{Number of Data Points Per Factor}\\
    & data_{n,d} && \text{n by d Matrix of Each Data Point}\\
    & rets_d && \text{Array Containing the Returns at Each Data Point}\\
    \end{aligned}$$

### 2.3. Variables

$$\begin{aligned}
    \text{Variables:}\qquad& b_i && i=1,\dots,n && \text{Weight Given to Each Factor}\\
    & yt_i && i=1,\dots,d && \text{Placeholder Variable for Logic Constraints}\\
    & t_i && i=1,\dots,n && \text{Absolute Value of } b_i\\
    & maxt && && \text{Largest } t_i \text{ Value}\\
    \end{aligned}$$

### 2.4. Constraints

$$\begin{aligned}
    \text{All Objective Functions Are Subject To:}\qquad& yt_i = b_i \cdot data[i,:] && i=1,\dots,d && \text{(1)}\\
    & t_i \ge b_i && i=1,\dots,n && \text{(2)}\\
    & t_i \ge -b_i && i=1,\dots,n && \text{(3)}\\
    & maxt \ge t_i && i=1,\dots,n && \text{(4)}\\
    \end{aligned}$$

The first constraint sets $yt_i$ equal to the variable $b_i$ multiplied (using dot product) to the data associated data from our generated CSV file.  This constraint esablishes the variable $yt_i$ to be compared to the returns data within our regression models below. 

Constraints (2) and (3) establish the variable $t_i$ as the absolute value of variable $b_i$ using logic.  Since $t_i$ must be greater than or equal to $b_i$ while still being greater than or equal to $-b_i$, the only value it can take is the possitive value of $b_i$.  We need these constraints and the variable $t_i$ for our LASSO regressions seen in objective functions 

The last constraint sets the variable maxt equal to the largest value of $t_i$ within our model.  This is necessary for running our L$_\infty$ regularization in our models below.

### 2.5. Objective Functions

$$\begin{aligned}
     & \underset{x \in \mathbb{R^n}}{\text{Minimize}}\qquad& f_1(x) = \sum_{i=1}^{d}(rets - yt_i)^2 + \lambda * maxt + \mu * \sum_{i=1}^{n}t_i && \text{(1) LASSO with } L_\infty \text{ Regularization}\\
     & \underset{x \in \mathbb{R^n}}{\text{Minimize}}\qquad& f_2(x) = \sum_{i=1}^{d}(rets - yt_i)^2 + \mu * maxt && \text{(2) } L_\infty \text{ Regularization}\\
     & \underset{x \in \mathbb{R^n}}{\text{Minimize}}\qquad& f_3(x) = \sum_{i=1}^{d}(rets - yt_i)^2 + \lambda * \sum_{t=1}^{n}t_i && \text{(3) LASSO} \\
     & \underset{x \in \mathbb{R^n}}{\text{Minimize}}\qquad& f_4(x) = \sum_{i=1}^{d}(rets - yt_i)^2 + \lambda * \sum_{b=1}^{n}b_i^2 && \text{(4) } L_2 \text{ Regularization}\\
     & \underset{x \in \mathbb{R^n}}{\text{Minimize}}\qquad& f_5(x) = \sum_{i=1}^{d}(rets - yt_i)^2  && \text{(5) Least Squares} \\
     \end{aligned}$$

Out of curiosity, we chose to analyze a number of different regression approaches in order to see how their results  differ when compared to one another.  In each case, the model's returns are compared against an array of returns to determine the difference in results. Explanations of each of our models can be found below.

(1) Our first objective funciton is a LASSO with $L_\infty$ regularization. The purpose of a lasso regression is to sparsify the solution. It does this by taking the sum of all of the coefficients for $b_i$ times a regularization parameter.  Doing this produces a smaller number of selected factors to be analyzed.  The purpose of an $L_\infty$ regression is to equalize the solution. This forces the coefficient values to all remain closer together.  By combining the LASSO and $L_\infty$ regression models we are sparsifying the number of factors we select while also keeping our coefficients closer to each other.

(2) Our second objective function is strictly an $L_\infty$ regularization. It has less of a correction for the coefficient values than the first model.

(3) Our third objective function is strictly the LASSO portion of the first model.

(4) The fourth objective function uses ridge regularization ($L_2$) to generate our list of useful factors.  The purpose of a ridge regression is to smooth the solution. This is done by taking the sum of each coefficient and squaring it then multiplying by a regularization parameter.

(5) Our fifth objective function uses a simple least squares model with no regularization parameters.

## 3. Solution ##


### Steve's Notes
Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [7]:
using JuMP, Clp

m = Model(solver = ClpSolver())

things = [:horses, :donkeys, :goats]  # these are the things
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
solve(m)

for i in things
    println("The total number of ", i, " is: ", getvalue(x[i]))     # print result
end

INFO: Precompiling module Clp.


The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


In [17]:
using JuMP, Gurobi

lambda = .1
mu = 5000

rets = readcsv("./returns.csv")
rets = rets[2:length(rets[:,1]), 2]

data = readcsv("./percent_changes.csv")
dates = data[2:length(data[:,1]), 1] #extract date column
data = data[:, 2:length(data[1,:])]
cols = data[1,:] #extract column titles
data = data[2:length(data[:,1]),:] #remove column titles

data = convert(Array{Float64},data)
rets = convert(Array{Float64},rets)

num_factors = length(data[1,:])
num_dates = length(dates)

m = Model(solver = GurobiSolver(OutputFlag = 0))

@variable(m, b[1:num_factors])
@variable(m, yt[1:num_dates])
@variable(m, t[1:num_factors])
@variable(m, maxt)
@constraint(m, [i in 1:num_dates], yt[i] == dot(b, data[i, :]))
@constraint(m, t .>= b)
@constraint(m, t .>= -b)
@constraint(m, [i in 1:num_factors], maxt >= t[i])
@objective(m, Min, sum( (rets - yt).^2 ) + lambda * maxt + mu * sum(t)) #LASSO with L-Infinite Regularization
# @objective(m, Min, sum( (rets - yt).^2 ) + mu * maxt) #L-Infinite Regularization
# @objective(m, Min, sum( (rets - yt).^2 ) + mu * sum(t)) #LASSO
# @objective(m, Min, sum( (rets - yt).^2 ) + lambda * sum(b.^2))
# @objective(m, Min, sum( (rets - yt).^2 ))

solve(m)

selection = []

bs = getvalue(b)
# bsort = sort(bs)
# for i in 1:length(bsort)
#     for j in 1:length(bs)
#         if bs[j] == bsort[i]
#             print(j, "\t", bsort[i],"\n")
#         end
#     end
# end
# print(count,"\n")
count = 0
for i in 1:length(bs)
    if bs[i] > 0.0000001
        print(bs[i], "\t", cols[i]+1,"\n")
        count += 1
        push!(selection, i)
    end
end
print(count,"\n")

sel_data = data[:, selection]
num_factors = length(sel_data[1,:])

m2 = Model(solver = GurobiSolver(OutputFlag = 0))

@variable(m2, b2[1:num_factors])
@variable(m2, yt2[1:num_dates])
@constraint(m2, [i in 1:num_dates], yt2[i] == dot(b2, sel_data[i, :]))
@objective(m2, Min, sum( (rets - yt2).^2 ) + lambda * sum(b2.^2))

solve(m2)

print("\n\n\n\n")
bs2 = getvalue(b2)
count = 0
for i in 1:length(bs2)
    if bs2[i] > 0.0000001
        print(bs2[i], "\t\n")
        count += 1
    end
end
print(count)

Academic license - for non-commercial use only
33.97731137497124	7
5.766601280303161	9
4.939021111330837	10
3.268006844609805	22
93.106144610311	29
20.11602039854394	40
39.963734450929906	48
4.198201468312401	61
8.118749765484605	62
45.74950756202005	64
52.569678690011244	72
54.24378924251289	74
65.96509616010478	75
19.77952925778222	77
124.81741892147932	78
68.67471325124713	79
67.98420256786969	80
9.041109925112698	85
65.56608226613841	88
25.29991772685001	90
7.55020183525953	92
22.39805858801727	95
15.010843265386955	98
34.35239164705637	99
24
Academic license - for non-commercial use only




32.939235110665194	
7.367228692905606	
34.10717313663386	
26.245330839433198	
66.12036297117199	
2.801779390681785	
61.75964995245457	
21.54772988801555	
20.78356538192992	
42.975216834843366	
58.27667849900985	
43.35414443188283	
36.01166942681837	
34.65441724421927	
142.16243083009704	
44.645003702299604	
39.28941505574677	
48.0560123615054	
101.26983265981849	
46.709368287611454	
26.3944118

In [39]:
data

1000×100 Array{Float64,2}:
  0.00204736  -0.135141    -0.107811     …  -0.0209075   -0.000109472
 -0.157469    -0.00915567  -0.0457541       -0.0340217   -0.0456843  
 -0.136031     0.0542648   -0.000262748      0.0276062    0.00842596 
 -0.0155871    0.062749    -0.00808564      -0.00299174  -0.0198909  
 -0.122152    -0.173265     0.0730716        0.0736405    0.0946953  
  0.0563802    0.00203149  -0.0232352    …  -0.00519397  -0.00964657 
 -0.0135168    0.217249    -0.0537929        0.00579626  -0.00440167 
  0.0114186   -0.108382    -0.0513474       -0.0868291   -0.0694285  
 -0.0128393   -0.152965    -0.0638119       -0.1873      -0.186537   
 -0.233902     0.0716506   -0.0845752       -0.0494206   -0.0651204  
  0.0580849    0.0181498   -0.0119948    …   0.0104717   -0.00358676 
  0.0990594    0.238115     0.0539379        0.169785     0.154893   
 -0.0723073   -0.0608562   -0.0309915       -0.0190185   -0.0234224  
  ⋮                                      ⋱                     

In [9]:
bs = getvalue(b)
count = 0
for i in 1:length(bs)
    if bs[i] > 0.0000001
        print(bs[i], "\t", cols[i],"\n")
        count += 1
    end
end
print(count)

0.001175924781466682	debt
0.001176379876447029	debtusd
0.0011763428882229532	ebitdausd
0.0011760615672569486	epsusd
0.00117567451945085	equityusd
0.0011763651023670068	evebit
0.0011763909143901509	grossmargin
0.00047536625290461675	investmentsc
0.0011762452646425403	ncfcommon
0.0011755777254313419	ncfi
0.00038442906607985056	ncfinv
0.0009771962461347118	netincdis
0.0011761911436200718	netincnci
0.0003095008901513142	payables
0.0011763990948827534	pe
0.00018843827921669152	prefdivis
0.0011675827525512285	revenueusd
4.163285033462269e-7	sharefactor
0.0005278349385322946	sps
0.0011754952358400995	taxexp
20

In [10]:
sel_data

7×20 Array{Float64,2}:
 -0.00103938   4.06405    0.156267    …  -1.14286  -4.03807    2.05337  
  0.30437      6.46051    0.00650794      0.0       0.245311   0.498002 
 -0.634001    -0.279962  -3.18678         0.0      -1.1883    -2.03745  
 -2.86162     -0.494445  -1.13603         0.0       2.1999     0.89586  
 -0.676859    -2.1649    -1.48907         0.0       1.08407   -0.244727 
  3.36258      1.34137   -0.886255    …   0.0      -0.399574  -1.8382   
  0.0807311   -0.166374  47.4322          0.0      -2.67365    0.0910096

In [11]:
num_factors

20

In [12]:
getobjectivevalue(m)

1.2888055051188396e-5

In [13]:
data[:,[1,20,37]]

7×3 Array{Float64,2}:
  0.0434216   0.106106   -0.716398
 -0.673924   -0.2201      4.23729 
 -2.1544     -0.597726   -0.476165
  1.75225    -1.36557     0.574577
 -0.70383     6.28081    -4.20583 
  0.141441   -0.0405702  -0.926255
 -0.386225    0.556266    0.91608 

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

 Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are          | Cool  |
| ------------- |:-------------| -----:|
| col 3 is      |right-aligned |\$1600 |
|  colons       | align columns|  \$12 |
| zebra stripes |    are neat  |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.